This is the final project "sandbox" for CSC6621 - Arsalon's Version


In [20]:
# imports 
import os
import subprocess
import shutil

# not required if using the Docker Image 
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator # type: ignore
import matplotlib.pyplot as plt # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
import numpy as np # type: ignore

import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [2]:
from pyunpack import Archive

# Define the path to the directory containing the split archive
archive_directory = 'data/img_celeba.7z'

# Specify the first file of the split archive
first_part_path = os.path.join(archive_directory, 'img_celeba.7z.001')

# Define the output directory for the extracted files
output_directory = 'data/img_celeba_extracted'

# Extract the archive
try:
    Archive(first_part_path).extractall(output_directory)
    print("Extraction successful.")
except Exception as e:
    print("Failed to extract:", e)



Extraction successful.


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the output directory from previous context
output_directory = 'data/img_celeba_extracted'

# Path to the directory where all images are stored
image_directory = os.path.join(output_directory, 'img_celeba')

# Define the train and test directories
train_dir = os.path.join(output_directory, 'train')
test_dir = os.path.join(output_directory, 'test')

# Create directories for train and test datasets if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get all file names in the output directory
all_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

# Split files into train and test sets (80-20 split here)
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

# Function to copy files to new directories
def copy_files(files, directory):
    for file in files:
        shutil.copy(file, directory)

# Copy files to respective directories
copy_files(train_files, train_dir)
copy_files(test_files, test_dir)

print("Files successfully split into training and testing directories.")


Files successfully split into training and testing directories.


ImageDataGenerator: 

-From Tensorflow helps performing operations such as rescaling, rotating, shifting, shearing, and flipping. 
-Reads images in large numbers from a directory and automatically feed them to the model in batches

In [26]:
# Define paths
train_dir = 'data/img_celeba_extracted/train'  
test_dir = 'data/img_celeba_extracted/test'    

# Load the labels dataset and ensure labels are string type
file_path = os.path.join(os.getcwd(), 'data', 'identity_CelebA.txt')
data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=['filename', 'label'])
data['label'] = data['label'].astype(str)  # Convert labels to string to match expected format

# Check file existence for train, test, and validate subsets
data['train_exists'] = data['filename'].apply(lambda x: os.path.exists(os.path.join(train_dir, x)))
data['test_exists'] = data['filename'].apply(lambda x: os.path.exists(os.path.join(test_dir, x)))

# Separate DataFrames for train, test based on file existence
train_data = data[data['train_exists']]
test_data = data[data['test_exists']]

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train Generator - updated class_mode to 'categorical'
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=(64, 64),  # Resizes all images to 64x64
    batch_size=32,
    class_mode='categorical',  
    subset='training'
)

# Validation Generator (from train data) - also 'categorical'
validation_generator = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Test Generator - also 'categorical'
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data,
    directory=test_dir,
    x_col='filename',
    y_col='label',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Print the shapes of input images and labels from the generator
inputs, labels = next(train_generator)
print('Input batch shape:', inputs.shape)
print('Label batch shape:', labels.shape)


Found 129664 validated image filenames belonging to 10153 classes.


/opt/homebrew/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 40520 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 32415 validated image filenames belonging to 10153 classes.


/opt/homebrew/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 40520 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 40520 validated image filenames belonging to 9362 classes.
Input batch shape: (32, 64, 64, 3)
Label batch shape: (32, 10153)


/opt/homebrew/lib/python3.11/site-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 162079 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [27]:
# Number of classes
num_classes = 10153

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Update for 10,177 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Number of epochs to train for
    validation_data=test_generator  # Validation data to evaluate the model
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test accuracy:", test_accuracy)



Epoch 1/10
2543/4052 [=================>............] - ETA: 3:18 - loss: 9.1952 - accuracy: 1.1060e-04

KeyboardInterrupt: 